In [23]:
from madhatter import *
from nltk.corpus import gutenberg
import pandas as pd
import spacy
import nltk

%load_ext memory_profiler


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [2]:
files = [gutenberg.raw(fileid)[:100_000] for fileid in gutenberg.fileids()]
# files = files*100

nlp = spacy.load("en_core_web_sm", disable=[
                 "ner", 
                #  "lemmatizer", 
                 "textcat", "attribute_ruler"])
nlp.pipe_names


['tok2vec', 'tagger', 'parser', 'lemmatizer']

In [6]:
# %%memit
docs = nlp.pipe(files, n_process=-1)
for doc in docs:
    sent = list(doc.sents)[0]
    for token in sent:
        # print(token.text, token.tag_, token.pos_, token.dep_, token.is_stop)
        pass


/Users/Matey/miniconda3/envs/mlkit/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [26]:
%%mprun -f x
# for file in files:
#     bench = CreativityBenchmark(file)
#     sent = bench.tagged_sents[0]
#     for tup in sent:
#         # print(word)
#         pass

def x():
    def do_work(file):
        bench = CreativityBenchmark(file)
        sent = bench.tagged_sents[0]
        for tup in sent:
            # print(word)
            pass
        print(f"work_done-{file[:20]}")
    
    # from multiprocessing import pool
    from multiprocess import pool

    p = pool.Pool(processes=8)
    res = p.map(do_work, files)
# with pool.Pool() as p:
#     res = p.map(do_work, files)

x()

ERROR: Could not find file <string>
work_done-[Poems by William Bl
work_done-[Sense and Sensibili
work_done-[Persuasion by Jane 
work_done-[The Adventures of B
work_done-[Emma by Jane Austen
work_done-[The King James Bibl
work_done-[Stories to Tell to 
work_done-[Alice's Adventures 
work_done-[The Ball and The Cr
work_done-[The Wisdom of Fathe
work_done-[The Parent's Assist
work_done-[The Man Who Was Thu
work_done-[Moby Dick by Herman
work_done-[Paradise Lost by Jo
work_done-[The Tragedie of Jul
work_done-[The Tragedie of Ham
work_done-[The Tragedie of Mac
work_done-[Leaves of Grass by 



### Memory usage of Spacy vs Custom Package
| Framework | peak memory | increment |
|  ------ | ---------- | -------- |
| Spacy | 5089.13 MiB |  4465.29 MiB |
| Mad Hatter| 434.81 MiB  | 48.75 MiB |

Increment here is the more important number as it tells us how memory usage peaks when performing a given operation.
